### 【 D0120_work_김현우 】
- 주__제 : 손글씨 데이터 분류
- 데이터 : mnist_train.csv
- 구__성 : 피쳐(hour) + 타겟(score)
- 학__습 : 지도학습 + 회귀
- 구__현 : 인공신경망

[1] 모듈 로딩 및 데이터 준비 <hr>

In [42]:
#%pip install visdom

In [ ]:
# [1-1] 모듈 로딩
import pandas as pd                 # 데이터 분석 및 처리용 모듈
import torch                        # 텐서 및 수치, 기본 함수용 모듈
import torch.nn as nn               # 인공신경망 관련 모듈
import torch.nn.functional as F     # 인공신경망 함수(AF, LF, MF) 관련 모듈
import torch.optim as optim         # 경사하강법 알고리즘으로 최적화 관련 모듈
from torch.optim import Adam        # Adam 최적화 알고리즘
from torch.utils.data import TensorDataset, DataLoader

from torchinfo import summary

# [1-2] 데이터 준비
dataDF = pd.read_csv("../DATA/mnist_train.csv")
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, 5 to 0.617
dtypes: int64(785)
memory usage: 59.9 MB


In [44]:
# [1-3] 데이터 -> Tensor 변환
# feature
featureDF = dataDF[dataDF.columns[1:]]
xTS = torch.tensor(featureDF.values/255, dtype=torch.float32)

# target
targetDF = dataDF[dataDF.columns[0:1]]
yTS = torch.tensor(targetDF.values.flatten(), dtype=torch.long) 

print(f"xTS : {xTS.shape}, yTS : {yTS.shape}")

xTS : torch.Size([10000, 784]), yTS : torch.Size([10000])


In [ ]:
# 데이터셋 생성
dataset = TensorDataset(xTS, yTS)
print(len(dataset))  # 10000

# DataLoader 생성 (배치 크기: 32, 셔플: True)
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print(f"배치 개수: {len(train_loader)}")
# 출력: 배치 개수: 313 (10000 / 32 = 312.5 → 313)

10000
배치 개수: 313


[2] 다중분류 모델 설계 <hr>

In [ ]:
# -------------------------------------------------------------------------------
#               입력수           퍼셉트론수/출력수           AF(활성화함수)
# -------------------------------------------------------------------------------
# 입력층           784                   784                
# 은닉층1          784                   128                    ReLU
# 은닉층2          128                    64                    ReLU
# 은닉층3           64                    32                    ReLU
# 출력층            32                    10                   Softmax  다중분류
# -------------------------------------------------------------------------------
# ★ Pytorch에는 입력층 클래스 존재 X -> 입력 텐서를 입력층으로 간주
# -------------------------------------------------------------------------------
class MultiClassModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hd1_layer = nn.Linear(784, 128)
        self.hd2_layer = nn.Linear(128, 64)
        self.hd3_layer = nn.Linear(64, 32)
        self.out_layer = nn.Linear(32, 10)

    def forward(self, x):
        out = F.relu(self.hd1_layer(x))
        out = F.relu(self.hd2_layer(out))
        out = F.relu(self.hd3_layer(out))
        out = self.out_layer(out) # CrossEntropyLoss 안에 Softmax 내장

        return out

In [47]:
model = MultiClassModel()
print(model)
# summary(model, input_size=(1,784))

MultiClassModel(
  (hd1_layer): Linear(in_features=784, out_features=128, bias=True)
  (hd2_layer): Linear(in_features=128, out_features=64, bias=True)
  (hd3_layer): Linear(in_features=64, out_features=32, bias=True)
  (out_layer): Linear(in_features=32, out_features=10, bias=True)
)


[3] 학습 준비 <hr>

In [48]:
# [3-1] 학습 설정
EPOCHS = 20
BATCH_SIZE = 32
COUNT = int(xTS.shape[0] / BATCH_SIZE)

# [3-2] 학습 인스턴스
loss_fn = nn.CrossEntropyLoss() # softmax, log 자동 적용
                                # softmax 는 모든 출력을 확률로 변환
adam_opt = Adam(model.parameters())

[4] 학습 진행<hr>

In [49]:
# 학습 진행
for epoch in range(EPOCHS + 1):
    total_loss = 0
    
    for idx in range(COUNT):
        # - 배치크기만큼 데이터 추출 인덱스
        sIdx = idx * BATCH_SIZE
        eIdx = sIdx + BATCH_SIZE
        
        # - 배치크기만큼 순전파 진행 ==> 예측값 추출
        pre_y = model(xTS[sIdx:eIdx])

        # - 손실계산
        loss = loss_fn(pre_y, yTS[sIdx:eIdx])
        
        # - 역전파 + 최적화
        adam_opt.zero_grad()
        loss.backward()     ##<- 역전파: 경사하강법으로 W,b 계산진행
        adam_opt.step()     ##<- 새로운 W,b 업데이트 진행
        
        # - 배치 크기 loss 누적
        total_loss += loss.item()
    
    if epoch % 5 == 0:  
        print(f'[{epoch:03d}_에포크] loss : {total_loss/COUNT:.5f}')

[000_에포크] loss : 0.75851
[005_에포크] loss : 0.10095
[010_에포크] loss : 0.02573
[015_에포크] loss : 0.02178
[020_에포크] loss : 0.01391


In [50]:
def evaluate_model(model, data_loader):
    model.eval()  # 평가 모드 (드롭아웃, 배치정규화 비활성화)
    
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():  # 그래디언트 계산 안 함 (메모리 절약)
        for x_batch, y_batch in data_loader:
            # 예측
            predictions = model(x_batch)  # (batch, 10)
            
            # 가장 높은 점수의 클래스 선택
            predicted_labels = torch.argmax(predictions, dim=1)  # (batch,)
            
            # 정답과 비교
            correct = (predicted_labels == y_batch).sum().item()
            total_correct += correct
            total_samples += y_batch.size(0)
    
    accuracy = total_correct / total_samples * 100
    print(f"Accuracy: {accuracy:.2f}%")
    
    return accuracy

In [51]:
# 평가 실행
evaluate_model(model, train_loader)

Accuracy: 99.63%


99.63